<h1><center> Text mining part </center></h1>

In [1]:
import requests
from bs4 import BeautifulSoup

In [43]:
add_page = requests.get('https://auto.bazos.cz/inzerat/156776866/skoda-octavia-3-fc-14tsi-cng-81kw-dsg-style-koupcr107tkm.php')

In [44]:
soup_add = BeautifulSoup(add_page.text, 'html')

In [45]:
add = soup_add.find('div', {'class':'popisdetail'}).get_text()
test = soup_add.find('table').find_all('b')[-1].get_text()
print(test)


  399 900 Kč


In [19]:
for child in soup_add.find('table').children
    print(child)



<tr><td class="listadvlevo" valign="top" width="35%">
<table cellpadding="0" cellspacing="1" width="*">
<tr><td height="22px" width="25%">Jméno:<br/><br/></td><td colspan="2"><b><a href="https://www.bazos.cz/hodnoceni.php?idmail=6108226&amp;idphone=3722517&amp;jmeno=Luk%C3%A1%C5%A1+Lau+" rel="nofollow">Lukáš Lau </a></b><br/><br/></td></tr>
<tr id="overlaydetail"><td height="22px" width="25%">Telefon:<br/><br/></td><td colspan="2"><span class="teldetail" onclick="odeslatrequest('/detailtel.php?idi=157061853&amp;idphone=3722517');"><span style="white-space: nowrap;">732... zobraz číslo</span></span><br/><br/></td></tr>
<tr><td height="22px" width="25%">Lokalita:</td><td width="9%"><img alt="Mapa" height="17" src="https://www.bazos.cz/obrazky/map.svg"/></td><td><a href="https://www.google.com/maps/place/49.864104,18.524323/@49.864104,18.524323,12z/data=!4m2!3m1!1s0x0:0x0" rel="nofollow" title="Přibližná lokalita">734 01</a> <a href="https://auto.bazos.cz/inzeraty/karvin%C3%A1-4/73401/"

In [46]:
def remove_spaces(text):
    return text.replace(" ", "").replace(".", "").replace("xxx", "000")

In [47]:
add = remove_spaces(add)
print(add)

000POZORNEJEDNÁSEODOVOZZEZAHRANIČÍ!000-nehrozírizikostočenéhotachometruaprávníchvadjakoudovozu!

Prodáminasplátky-ŠkodaOctaviacombi14TSiG-TECCNGSTYLEKoupČR,1majitel,Servisníkniha

PropodnikateleafirmyodpočetDPH!!!

000smožnostíZÁRUKYažna24měsíců!!!000-ojetývůzseZÁRUKOUjakonaNOVÝ!!!

000POHONCNGaplnohodnotnánádržnaBENZÍN!!!
XXXautomatickápřevodovkaDSG!XXX

rv2018
81kW-110koní
najeto-107100km
PravidelnýservispouzeautorizovanýservisŠkoda
bílá

autpřevodovka,airbagřidiče,ABS,stabilizacepodvozku(ESP),protiprokluzovýsystémkol(ASR),autzabrzděnívkopci,autklimatizace,tempomat,alukola,palubnípočítač,parkovacíasistent,multifunkčnívolant,deaktivaceairbaguspolujezdce,elokna,střešnínosič,elzrcátka,startovánítlačítkem,imobilizér,alarm,centráldálkový,vyhřívanásedadla,výškověnastavitelnásedadla,výškověnastavitelnésedadlořidiče,senzortlakuvpneumatikách,zadnísvětlaLED,ostřikovačesvětlometů,mlhovky,autorádio,vyhřívanázrcátka,klimatizovanápřihrádka,dělenázadnísedadla,USB,handsfree,dvouzónováklimatizace,blu

In [48]:
def get_numbers_from_text(text):
    import re
    text = remove_spaces(text)
    pattern = '[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?'
    list_of_numbers = re.findall(pattern, text)
    return list_of_numbers

In [49]:
all_numbers = get_numbers_from_text(add)
print(all_numbers)
type(all_numbers[0])

['000', '000', '14', '1', '000', '24', '000', '000', '2018', '81', '-110', '-107100', '7', '1', '100', '36', '10', '-739266184', '15', '89', '2012', '3558']


str

In [38]:
def find_years(numbers):
    numbers = [x for x in numbers if (float(x) > 1980) and (float(x) < 2030)]
    return numbers

In [39]:
all_years = find_years(all_numbers)
print(all_years)

['2022', '2022', '2015']


In [40]:
type(all_years[0])

str

In [40]:
def find_km(numbers):
    numbers = [x for x in numbers if (float(x) > 3000) and (float(x) < 500000)]
    return numbers

In [41]:
test = find_km(all_numbers)
print(test)

['92000', '0195566']


In [42]:
def get_context(text, list_of_tokens, year_dictionary, km_dictionary, context_span=20):
    import re
    context = []
    year = 'No match'
    km = 'No match'
    for token in find_years(list_of_tokens):
        all_occurences_indices = [m.start() for m in re.finditer(token, text)]
        for index in all_occurences_indices:
            left_index = max(index - context_span, 0)
            right_index = min(index + context_span, len(text))
            substring = text[left_index: right_index].strip()
            for s in year_dictionary:
                year_find = [m.start() for m in re.finditer(s, substring)]
                if len(year_find) > 0:
                    year = token
    for token in find_km(list_of_tokens):
        all_occurences_indices = [m.start() for m in re.finditer(token, text)]
        for index in all_occurences_indices:
            left_index = max(index - context_span, 0)
            right_index = min(index + context_span, len(text))
            substring = text[left_index: right_index].strip()
            for s in km_dictionary:
                km_find = [m.start() for m in re.finditer(s, substring)]
                if len(km_find) > 0:
                    km = token
    return [year, km] #context
year_dict = ['egistr', 'rv', 'yrob', 'výrob', 'prov']
km_dict = ['km', 'Km', 'KM', 'ilomet', 'ajet', 'ájez', 'achom', 'atoč']
context_got = get_context(add, all_numbers, year_dict, km_dict)
print(context_got)

['2015', '92000']


In [52]:
def get_context(text, list_of_tokens, context_span=20):
    import re
    context = []
    for token in list_of_tokens:
        all_occurences_indices = [m.start() for m in re.finditer(token, text)]
        for index in all_occurences_indices:
            left_index = max(index - context_span, 0)
            right_index = min(index + context_span, len(text))
            substring = text[left_index: right_index].strip()
            
            if left_index == 0:
                substring = ' '.join(substring.split(' ')[:-1])
            elif right_index == len(text):
                substring = ' '.join(substring.split(' ')[1:])
            else:
                substring = ' '.join(substring.split(' ')[1:-1])
            context.append(substring)
    return context
context_got = get_context(add, all_years)
print(context_got)

['', '']


In [54]:
def find_year_of_manifacture(contexts):
    import re
    for text in contexts:
        test = [m.start() for m in re.finditer('egist', text)]
        if len(test) > 0:
            year = 
    return km_number    
find_year_of_manifacture(context_got)

['76699', '2', '2018', '107604']

In [51]:
def find_km(contexts):
    import re
    for text in contexts:
        test = [m.start() for m in re.finditer('Km', text)]
        if len(test) > 0:
            km_number = get_numbers_from_text(text)
    return km_number    
find_km(context_got)

['202160']

In [43]:
def get_context(text, list_of_tokens, context_span=20):
    import re
    context = []
    all_indexes = []
    for token in list_of_tokens:
        all_occurences_indices = [m.start() for m in re.finditer(token, text)]
        all_occurences_indices_ends = [m.end() for m in re.finditer(token, text)]
        
        if len(all_occurences_indices) > 1:
            all_occurences_indices = [max(all_occurences_indices)] #have to solve more sophistically
        if len(all_occurences_indices_ends) > 1:
            all_occurences_indices_ends = [max(all_occurences_indices_ends)]
        all_indexes.append(all_occurences_indices_ends)
        print(max(all_indexes))
        print(all_indexes[-1])
        
        for index in all_occurences_indices:
            print(index)
            left_index = max(min(index - context_span, index - all_indexes[-1]), 0)
            #left_index = max(index - context_span, 0)
            right_index = min(index + context_span, len(text))
            substring = text[left_index: right_index].strip()
            
            if left_index == 0:
                substring = ' '.join(substring.split(' ')[:-1])
            elif right_index == len(text):
                substring = ' '.join(substring.split(' ')[1:])
            else:
                substring = ' '.join(substring.split(' ')[1:-1])
            context.append(substring)
    return context, all_indexes
get_context(add, all_numbers)

[24]
[24]
21


TypeError: unsupported operand type(s) for -: 'int' and 'list'